# This notebook contains song information obtained via web scraping, retrieves lyrics through genius API, and manual correction. the two resulting lists of dictionaries are stored as json files and utilized in subsequent notebooks.

# Collecting and organizing your corpus

## COMM313 Spring 2020


### DUE FRIDAY APRIL 17 at 11.59PM (EST)



<div style="font-size:14pt; line-height: 1.5; ">
 <p style="font-size:16pt">The goal this assignment if for you to collect a pilot sample of the data for your final project and carry out some initial analyses on these data.</p>
 
 <ul>
     <li>There are a series of questions that make sure you have
         <ol>
        <li>collected some data</li>
        <li>started to clean and organize these data</li>
    <li>have begun to do some initial analyses on your pilot data</li>
         </ol>
    </li>

 </ul>
 
 
</div>


## Setup

* Add any additional modules you need here

In [1]:
import pandas as pd
from collections import Counter
import os
import json
import requests
from bs4 import BeautifulSoup
import lyricsgenius
import re

import nltk
from nltk import Text
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Commjhub/jupyterhub/comm
[nltk_data]     318_fall2019/nelkassabany/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Functions

* Add any functions from previous noteboooks you might need here


* REMEMBER you can include the functions in another notebook (e.g. the `functions.ipynb` in this folder) using:
    ```
    %run functions.ipynb
    ```
    
  You can add your own notebook with the functions you need as well or just add them in a series of code cells in this notebook.

In [2]:
def tokenize(text, lowercase=False, strip_chars=''):
    '''create a list of tokens from a string by splitting on whitespace and applying optional normalization 
    
    Args:
        text        -- a string object containing the text to be tokenized
        lowercase   -- should text string be normalized as lowercase (default: False)
        strip_chars -- a string indicating characters to strip out of text, e.g. punctuation (default: empty string) 
        
    Return:
        A list of tokens
    '''
    
    # create a replacement dictionary from the
    # string of characters in the **strip_chars**
    rdict = str.maketrans('','',strip_chars)
    
    if lowercase:
        text = text.lower()
    
    tokens = text.translate(rdict).split()
    
    return tokens

In [3]:
def get_ngram_tokens(tokens, n=1):
    '''create a list of n-gram tokens from a list of tokens
    
    Args:
        tokens -- a list of tokens
        n      -- the size of the window to use to build n-gram token list
        
    Returns:
        
        list of n-gram strings (whitespace separated) of length n
    '''
    
    if n<2 or n>len(tokens):
        return tokens
    
    new_tokens = []
    
    for i in range(len(tokens)-n+1):
        new_tokens.append(" ".join(tokens[i:i+n]))
        
    return new_tokens

In [4]:
#technically this works, but bc it relies on what's in the dictionary
## it doesn't accommodate discrepancies between 
def search_individual_song(songtitle, dictionary):
    iso_dict = next((song for song in dictionary if song['song_title'] == songtitle), None)
    song = genius.search_song(iso_dict['song_title'], iso_dict['performing_artist'])
    iso_dict['lyrics'] = song.lyrics
    
    return iso_dict

## 1. Describe how you have started to collect the corpus/corpora needed for your project

* Be specific about:
    * the keyword search, date range and sources if you are collecting a news-based corpus
    * the keywords, hashtags, mentiond etc, date ranges for a Twitter search
    * the websites and methods used if you are building your corpus from a web source (e.g. used billboard100 to get list of titles

using the table at this URL (https://en.wikipedia.org/wiki/Grammy_Award_for_Song_of_the_Year) I scraped the award year, title of winning song, and its performing artist into a list of dictionaries. then using the genius API, i obtained the lyrics for these songs. for the full corpus, i plan on creating another list of dictionaries of "losers" scraped from the nominees column in the table and genius api again to obtain the lyrics. 
update - starting to think i might need more data, perhaps do similar scrape of RECORD of the year winners/nominees. find info about sales/billboard placements of these songs to understand qualities of winning songs (how much does composition/engineering/commercial success contribute to the win, not just songwriting)

## 2. Pilot data


* If you haven't uploaded your data yet place your pilot data into the `data` folder


* Show a listing of the pilot data files (use `os.listdir()`)

# scraping for song titles, artists, award year

### winners

In [5]:
# YOUR CODE HERE
# 1. use the requests.get function to get retrieve your chart webpage

url = 'https://en.wikipedia.org/wiki/Grammy_Award_for_Song_of_the_Year'

page = requests.get(url)

# 2. get the html of the response and call it html
#
html = page.text
len(html)

# 3. create a document object using BeautifulSoup from html and call it doc
#    the code will look like:
#                             doc = BeautifulSoup(html, 'lxml')
doc = BeautifulSoup(html, 'lxml')

# 4. use the find() and find_all() search functions on this object to:
#    a. find the <table> element for the chart
#    b. from this element find all the <tr> elements nested in the <tbody>
#       and call your result rows


#have to use find_next_sibling bc the first table on the page is the summary info table at the top
rows = doc.find('table').find_next_sibling('table').find('tbody').find_all('tr')

type(rows)
#print(rows[20])
#print(rows[21])

bs4.element.ResultSet

In [6]:
# YOUR CODE HERE
winners_chart=[]


# 1. loop over the relevant items in rows (Note the first item is a header row!)

# 2. for each row extract the 
#      * year
#      * song
#      * PERFORMING artist
#    from the <td> (or <th>) elements

# 3. create a dictionary with keys 'year', 'song' & 'performing artist'

# 4. add this dictionary to the list called winners_chart

for row in rows[1:]:
    #extraction
    
    try:
        year = row.find('th').text.strip('\n')
        
    except:AttributeError
    pass
    
    #song title and artist are found in td element
    try:
        winner = row.find('td').text.replace('\n', " ")
        song_title = row.find('td').find_next_sibling('td').find_next_sibling('td').text.strip('"" \n * "')
    
        performing_artist = row.find('td').find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').text.strip('\n')
        
    except: AttributeError
    pass
  
    
    #dictionary
    dict = {
        'winner': winner,
        'year' : year,
        'song_title': song_title,
        'performing_artist': performing_artist
    }
    
    #add dict to chart list
    winners_chart.append(dict)


In [7]:
#winners_chart

In [8]:
#print(winners_chart)
# the year 1978 has 2 winners so this entry won't work with genius API. 
# i will separate them into 2 separate dictionaries, one for each song. this will give me 63 total entries in winner list

# 1961 winner was the Theme of Exodus, an instrumental track. the genius search returned the 
# ENTIRE TEXT of Thomas Aquinas Summa Contra Gentiles??? deleting that...


In [9]:
winners_chart[19]

{'performing_artist': 'Barbra Streisand',
 'song_title': 'Evergreen (Love Theme from A Star Is Born)',
 'winner': 'Barbra StreisandPaul Williams ',
 'year': '1978'}

In [10]:
winners_chart[20]

{'performing_artist': 'Barbra Streisand',
 'song_title': 'Debby Boone',
 'winner': 'Joe Brooks ',
 'year': '1978'}

In [11]:
winners_chart[20]['winner'] = "Joe Brooks"
winners_chart[20]['performing_artist'] = 'Debby Boone'
winners_chart[20]['song_title'] = 'You Light Up My Life'

len(winners_chart)
print(winners_chart[19:21])

[{'winner': 'Barbra StreisandPaul Williams ', 'year': '1978', 'song_title': 'Evergreen (Love Theme from A Star Is Born)', 'performing_artist': 'Barbra Streisand'}, {'winner': 'Joe Brooks', 'year': '1978', 'song_title': 'You Light Up My Life', 'performing_artist': 'Debby Boone'}]


### nominees

<div class="alert alert-info">
    <p>The string for the nominees follows the pattern:
    <pre>
    (Nominee[s]) for "(Title)" performed by (Artists)
    </pre>
    We can use a string matching pattern with a regular expression
    </p>
</div>

In [12]:
example_nom_str = 'Alan Jay Lerner & Frederick Loewe for "Theme to Gigi" performed by Various Artists'

In [13]:
re.match('(.*) for [\'"](.*)[\'"] performed by (.*)', example_nom_str).groups()

('Alan Jay Lerner & Frederick Loewe', 'Theme to Gigi', 'Various Artists')

* This returns a tuple where we can extract the components into the dictionary

In [14]:
nom_str_re = re.compile(r'(.*) for [\'"](.*)[\'"] (?:versions )?performed by (.*)')


noms = []

# 1. loop over the relevant items in rows (Note the first item is a header row!)

# 2. for each row extract the 
#      * year
#      * song
#      * PERFORMING artist
#    from the <td> (or <th>) elements

# 3. create a dictionary with keys 'year', 'song' & 'performing artist'

# 4. add this dictionary to the list called noms5

for row in rows[1:]:
    #extraction
    try:
        year = row.find('th').text.strip('\n')
    except:AttributeError
    pass
    
    
    ###PLAYING W LOOP####
    #cell = doc.find('table').find_next_sibling('table').find('tbody').find('ul')

    for sibling in row.find_all('li'):
        nom_str = sibling.text.strip('\n')
        try:
            nominees, song, artist = nom_str_re.match(nom_str).groups()
        except:
            nominees, song, artist = re.match('(.*) [\'"](.*)[\'"] (?:versions )?performed by (.*)', nom_str).groups()
        dict = {
        'year' : year,
        'nominees': nominees,
        'song_title': song,
        'performing_artist': artist}    
  
    #add dict to chart list
        noms.append(dict)


In [15]:
#noms

# retrieving lyrics

In [16]:
MY_CLIENT_ACCESS_TOKEN='cyS6-zlhQnNBmBXJARK-gtxTkuduVWS1M2RbZVHP3eOMsoolnOgRpg_x7K7rL3xX'

genius = lyricsgenius.Genius(MY_CLIENT_ACCESS_TOKEN)

In [17]:
error_processing=[]

# YOUR CODE HERE
for song_dict in winners_chart:
    try:
        song = genius.search_song(song_dict['song_title'], song_dict['performing_artist']) #retrieving lyrics
        song_dict['lyrics'] = song.lyrics #adding to dictionary
    
    except AttributeError: 
        print('no results found')
        error_processing.append(song_dict['song_title'])

    
if len(error_processing)>0:
    print('There were {} items in your chart that genius.com could not find'.format(len(error_processing)))

Searching for "Volare" by Domenico Modugno...
Done.
Searching for "The Battle of New Orleans" by Johnny Horton...
Done.
Searching for "Theme of Exodus" by Instrumental (Various Artists)...
Done.
Searching for "Moon River" by Henry Mancini...
Done.
Searching for "What Kind of Fool Am I?" by Sammy Davis Jr....
Done.
Searching for "Days of Wine and Roses" by Henry Mancini...
Done.
Searching for "Hello, Dolly!" by Louis Armstrong...
Done.
Searching for "The Shadow of Your Smile" by Tony Bennett...
Done.
Searching for "Michelle" by The Beatles...
Done.
Searching for "Up, Up, and Away" by The 5th Dimension...
Done.
Searching for "Little Green Apples" by O. C. Smith...
Done.
Searching for "Games People Play" by Joe South...
Done.
Searching for "Bridge over Troubled Water" by Simon & Garfunkel...
Done.
Searching for "You've Got a Friend" by James Taylor & Carole King...
Done.
Searching for "The First Time Ever I Saw Your Face" by Roberta Flack...
Done.
Searching for "Killing Me Softly with His

In [18]:
#print(winners_chart[2])
#the 1961 winner, Theme of Exodus, is an instrumental track, but the genius search returned 
# the ENTIRE TEXT of Thomas Aquinas Summa Contra Gentiles as lyrics??? deleting those

In [19]:
winners_chart[2]['lyrics'] = 'none'

In [20]:
#winners_chart

In [21]:
## manual fix for song
apples = next((song for song in winners_chart if song['song_title'] == 'Little Green Apples'), None)
apples['lyrics'] = open('data/little_green_apples_lyrics.txt').read()

### retrieving lyrics for nominees : 

In [22]:
for song_dict in noms:
    try:
        song = genius.search_song(song_dict['song_title'], song_dict['performing_artist']) #retrieving lyrics
        song_dict['lyrics'] = song.lyrics #adding to dictionary
    
    except AttributeError: 
        print('no results found')
        error_processing.append(song_dict['song_title'])
        song_dict['lyrics'] = 'no results found'

    
if len(error_processing)>0:
    print('There were {} items in your chart that genius.com could not find'.format(len(error_processing)))

Searching for "Catch a Falling Star" by Perry Como...
Done.
Searching for "Fever" by Peggy Lee...
Done.
Searching for "Theme to Gigi" by Various Artists...
Done.
Searching for "Witchcraft" by Frank Sinatra...
Done.
Searching for "High Hopes" by Frank Sinatra...
Done.
Searching for "I Know" by Perry Como...
Done.
Searching for "Like Young" by André Previn...
Done.
Searching for "Small World" by Johnny Mathis...
Done.
Searching for "He'll Have to Go" by Jim Reeves...
Done.
Searching for "Nice 'n' Easy" by Frank Sinatra...
Done.
Searching for "The Second Time Around" by Andy Williams...
Done.
Searching for "Theme from A Summer Place" by Percy Faith...
Done.
Searching for "Big Bad John" by Jimmy Dean...
Done.
Searching for "A Little Bitty Tear" by Burl Ives...
Done.
Searching for "Lollipops and Roses" by Jack Jones...
Done.
Searching for "Make Someone Happy" by Various Artists...
Done.
Searching for "As Long as He Needs Me" by Shirley Bassey...
Done.
Searching for "I Left My Heart in San F

Done.
Searching for "Vision of Love" by Mariah Carey...
Done.
Searching for "Another Day in Paradise" by Phil Collins...
Done.
Searching for "Nothing Compares 2 U" by Sinéad O'Connor...
Done.
Searching for "Hold On" by Wilson Phillips...
Done.
Searching for "(Everything I Do) I Do It for You" by Bryan Adams...
Done.
Searching for "Baby Baby" by Amy Grant...
Done.
Searching for "Losing My Religion" by R.E.M....
Done.
Searching for "Walking in Memphis" by Marc Cohn...
Done.
Searching for "Achy Breaky Heart" by Billy Ray Cyrus...
Done.
Searching for "Beauty and the Beast" by Celine Dion & Peabo Bryson...
Done.
Searching for "Constant Craving" by k.d. lang...
Done.
Searching for "Save the Best for Last" by Vanessa Williams...
Done.
Searching for "The River of Dreams" by Billy Joel...
Done.
Searching for "I'd Do Anything for Love (But I Won't Do That)" by Meat Loaf...
Done.
Searching for "If I Ever Lose My Faith in You" by Sting...
Done.
Searching for "Harvest Moon" by Neil Young...
Done.
S

In [23]:
error_processing

["Raindrops Keep Fallin' on My Head",
 'I.G.Y. (What a Beautiful World)',
 'I Can Love You Like That']

## manual fixes to no results and incorrect results

In [24]:
search_one = next((song for song in noms if song['song_title'] == "Raindrops Keep Fallin' on My Head"), None) 
song = genius.search_song("Raindrops Keep Fallin' on My Head", "B.J. Thomas")
search_one['lyrics'] = song.lyrics

print(search_one)

Searching for "Raindrops Keep Fallin' on My Head" by B.J. Thomas...
Done.
{'year': '1970', 'nominees': 'Burt Bacharach & Hal David', 'song_title': "Raindrops Keep Fallin' on My Head", 'performing_artist': 'B. J. Thomas', 'lyrics': "Raindrops keep fallin' on my head\nAnd just like the guy whose feet are too big for his bed\nNothin' seems to fit\nThose raindrops are fallin' on my head, they keep fallin'\n\nSo I just did me some talkin' to the sun\nAnd I said I didn't like the way he got things done\nSleepin' on the job\nThose raindrops are fallin' on my head, they keep fallin'\n\nBut there's one thing I know\nThe blues they send to meet me won't defeat me\nIt won't be long 'til happiness steps up to greet me\n\nRaindrops keep fallin' on my head\nBut that doesn't mean my eyes will soon be turnin' red\nCryin's not for me\n'Cause I'm never gonna stop the rain by complainin'\nBecause I'm free\nNothing's worryin' me\n\nIt won't be long 'til happiness steps up to greet me\n\nRaindrops keep fal

In [25]:
search_two = next((song for song in noms if song['song_title'] == 'I.G.Y. (What a Beautiful World)'), None)
## the only version on genius is by howard jones, which is a cover of the original grammy-nominated version.
## re-doing search with different artist name

In [26]:
search_two['lyrics'] = genius.search_song("I.G.Y (What a Beautiful World)", "Howard Jones").lyrics

Searching for "I.G.Y (What a Beautiful World)" by Howard Jones...
Done.


In [27]:
search_three = next((song for song in noms if song['song_title'] == 'I Can Love You Like That'), None)
## this nominated version is for All-4-One covering this song originially performed by john michael montgomery
## wikipedia lists them both as "performers" while genius has identical lyrics listed separately for the artists

search_three['lyrics'] = genius.search_song("I Can Love You Like That", "All-4-One").lyrics

Searching for "I Can Love You Like That" by All-4-One...
Done.


In [28]:
gigi = next((song for song in noms if song['song_title'] == 'Theme to Gigi'), None)
## original genius search result looks like the transcript of a talk abt DJing at SXSW
## this song is from a movie, so fetching lyrics manually

gigi['lyrics'] = open('data/gigi_lyrics.txt').read()
gigi

{'lyrics': "There's sweeter music when she sings, isn't there?\nA different bloom about her cheeks, isn't there?\nCould I be wrong, could it be so?\nOh where oh where did Gigi go?\nGigi, am I a fool without a mind?\nOr have I merely been too blind?\nTo realize.\nOh, Gigi!\nWhy, you've been growing up before my eyes\nGigi!\nYou're not at all that funny\nAwkward little girl I knew\nOh no!\nOvernight, there's been a breathless change\nIn you.\nOh, Gigi!\nWhile you were trembling on the brink\nWas I out yonder somewhere\nBlinking at a star?\nOh, Gigi!\nHave I been standing up too close?\nOr back too far?\nWhen did your sparkle turn to fire?\nAnd your warmth become desire?\nOh what miracle has made you the way you are?\nGigi!\nGigi!\nGigi!\nOh no!\nI was mad not to have seen the change\nIn you.\nOh Gigi!\nWhile you were trembling...are?",
 'nominees': 'Alan Jay Lerner & Frederick Loewe',
 'performing_artist': 'Various Artists',
 'song_title': 'Theme to Gigi',
 'year': '1959'}

In [29]:
like_young = next((song for song in noms if song['song_title'] == 'Like Young'), None)
## the original result was a chunk of les miserables! this is an instrumental track, deleting 

like_young['lyrics'] = 'none'
like_young

{'lyrics': 'none',
 'nominees': 'Paul Francis Webster & André Previn',
 'performing_artist': 'André Previn',
 'song_title': 'Like Young',
 'year': '1960'}

In [30]:
## make someone happy is a song that appeared in the musical do re mi, recorded by other artists in later years
## using one of these names to get the lyrics
happy = next((song for song in noms if song['song_title'] == 'Make Someone Happy'), None)
happy['lyrics'] = genius.search_song("Make Someone Happy", "Jamie Cullum").lyrics

Searching for "Make Someone Happy" by Jamie Cullum...
Done.


In [31]:
## the sweetest sounds appeared in the musical no strings. listed on genius under songwriter richard rodgers' name
sweetest = next((song for song in noms if song['song_title'] == 'The Sweetest Sounds'), None)
sweetest['lyrics'] = genius.search_song("The Sweetest Sounds", "Richard Rodgers").lyrics
#print(sweetest)

Searching for "The Sweetest Sounds" by Richard Rodgers...
Done.


In [32]:
## (i never promised you a) rose garden is listed without the parentheticals on genius
rose = next((song for song in noms if song['song_title'] == "(I Never Promised You a) Rose Garden"), None)
rose['lyrics'] = genius.search_song("Rose Garden", "Lynn Anderson").lyrics

Searching for "Rose Garden" by Lynn Anderson...
Done.


In [33]:
## just the two of us was scraped as "grover wash.. WITH bill withers" different than genius
two = next((song for song in noms if song['song_title'] == "Just the Two of Us"), None)
two['lyrics'] = genius.search_song("Just the Two of Us", "Grover Washington Jr.").lyrics

Searching for "Just the Two of Us" by Grover Washington Jr....
Done.


In [34]:
## how do i live is a good example of definition of song of the year. the nominee was dianne warren who wrote the song.
## leann rimes and trisha yearwood both recorded versions around the same time
live = next((song for song in noms if song['song_title'] == "How Do I Live"), None)
live['lyrics'] = genius.search_song("How Do I Live", "LeAnn Rimes").lyrics
print(live)

Searching for "How Do I Live" by LeAnn Rimes...
Done.
{'year': '1998', 'nominees': 'Diane Warren', 'song_title': 'How Do I Live', 'performing_artist': 'LeAnn Rimes & Trisha Yearwood', 'lyrics': "[Verse 1]\nHow do I get through one night without you?\nIf I had to live without you\nWhat kind of life would that be?\n\n[Pre-Chorus]\nOh, I, I need you in my arms, need you to hold\nYou're my world, my heart, my soul and if you ever leave\nBaby you would take away everything good in my life\nAnd tell me now\n\n[Chorus]\nHow do I live without you? I want to know\nHow do I breathe without you if you ever go?\nHow do I ever, ever survive?\nHow do I, how do I, oh, how do I live?\n\n[Verse 2]\nWithout you there'd be no sun in my sky\nThere would be no love in my life\nThere'd be no world left for me\n\n[Pre-Chorus 2]\nAnd I, baby, I don't know what I would do\nI'd be lost if I lost you, if you ever leave\nBaby, you would take away everything real in my life\nAnd tell me now\n\n[Chorus]\nHow do I l

In [35]:
## another issue of featured artists in wikipedia listing but not in genius
lights = next((song for song in noms if song['song_title'] == "All of the Lights"), None)
lights['lyrics'] = genius.search_song("All of the Lights", "Kanye West").lyrics

Searching for "All of the Lights" by Kanye West...
Done.


In [36]:
#noms

* Describe how you have or think you will organize your data
    * What are the dimensions of your analysis (e.g. _date_, _source_, etc.)
    * What are the size of units within these dimensions (e.g. _days_, _years_, _national v local_, _conservative vs liberal_ etc.)
    * What do you think is the best way to physically organize your corpus/corpora to support this (e.g. separate folders, filename naming scheme etc)

Not entirely sure if I'm understanding this correctly, but the primary dimension of analysis is award status, correct? win/did not win, which I'm planning to organize in two separate dictionaries. Time by years could be another dimension.
Since I'm collecting data via web scraping and genius API, all the necessary text data will contained within the lists of song dictionaries. one dictionary for "winning" songs and one for the rest of the nominees. 

## 3. Pilot data descriptives

* Write some code to describe your data according the dimensions you've chosen

    * e.g. if you have your data in folders by year (e.g. `data/1999`, `data/2000`, etc.) and a single text per file within each folder you could do something like:
        ```
        for year in os.listdir('data'):
            num_of_docs = len(os.listdir(os.path.join('data',year))
            print('{}\t{} texts'.format(year, num_of_docs))
        ```
            
* Do this for each of the relevant dimensions for your analysis.        

In [37]:
# YOUR CODE HERE

#pilot data just is just the works that won SOTY
print('there are {} winning songs'.format(len(winners_chart)))

print('there are {} non-winning/nominated songs'.format(len(noms)))



there are 63 winning songs
there are 258 non-winning/nominated songs


## JSON dumping the two corpora

In [38]:
# winning songs
with open('data/winning_song_corpus.json','w') as out:
    out.write(json.dumps(winners_chart))

In [41]:
# nominees
with open('data/nominees_corpus.json','w') as out:
    out.write(json.dumps(noms))

## 4. Pilot data analysis

* Add a series of code cells below showing some initial text analysis

    * This could include:
        1. creating word and n-gram frequency lists
        2. creating keyword lists by comparing different dimensions
        3. some targetted KWIC analyses
        4. collecting collocates of specific keywords
        5. scoring texts using VADER or other dictionary resources

Sorry this is so sparse! I'm catching up on the course material but wanted to submit something for feedback as I keep working.

In [39]:
win_word_freq = Counter()
win_bigram_freq = Counter()

remove = '()[]:;.-!?,'

for song in range(len(winners_chart)):
    lyrics_raw = winners_chart[song]['lyrics']
    
    song_toks = []
    temp = tokenize(lyrics_raw, lowercase = True, strip_chars = remove)
    for tok in temp:
        if tok not in stop_words:
            song_toks.append(tok)
    
    win_word_freq.update(song_toks)
    
    bigrams = get_ngram_tokens(song_toks, 2)
    win_bigram_freq.update(bigrams)
    
    #clean string
    #tokenize
    #run thru counter
    
#win_word_freq.most_common(75), win_bigram_freq.most_common(50)

In [40]:
nom_word_freq = Counter()
nom_bigram_freq = Counter()
remove = '()[]:;.-!?,'

for song in range(len(noms)):
    lyrics_raw = noms[song]['lyrics']
    song_toks = tokenize(lyrics_raw, lowercase = True, strip_chars = remove)
    
    nom_word_freq.update(song_toks)
    
    bigrams = get_ngram_tokens(song_toks, 2)
    nom_bigram_freq.update(bigrams)
    
    #clean string
    #tokenize
    #run thru counter
    
#nom_word_freq.most_common(75), nom_bigram_freq.most_common(50)

## 5. Questions/Problems/Requests for specific help in coding or organizing data 

* when it comes to collecting my full set of data, is it worth pulling pull other information about winning songs such as winners of the award (usually songwriters in addition to/independent of performing artist)? i think it could be interesting to see distribution of nationalities and how much overlap there is between the official listed winners and performer of the song (aka did they write it themselves, how much help did they have), but this would only apply for winners. scraping the nominees from this same table wouldn't give me the same amount of info
* in the case of wanting to see if songwriter = performing artist, is there a way to work around obstacle of different names in the credits
* for organizing the data, does it make more sense to make one dictionary of winning songs and another with the rest of the nominees, or one large dictionary with an "award status" key? values win/lose

* what is the best way to have separated the double winners for the 1978 year award? the "brute force" way that I did it was only possible bc the list was short enough to catch it
* is there anything to do with songs that are not in english or instrumental tracks other than eliminate them from analysis? i'm looking at the 1959 winner in italian and the 1961 winner which is instrumental
* when I try to use use list indices in defining iterations of a for loop [0: whatever], it returns a syntax error underneath the colon? not understanding why
* there was the thomas aquinas lyric mistake that i corrected again with brute force, but i think there are more errors from the genius search because words like "usage" "english" and "language" are coming up in the most common tokens. is there a good way to locate these within the list of dictionaries?